In [3]:
import pandas as pd
import 

In [16]:
#df = pd.read_excel("data/survey_data/labelled_data/sat_data_combined_vertical.xlsx")


In [17]:
df

,response,annotator1_score,annotator2_score,annotator3_score,empathy_score
0,When you've selected the protocol you want to ...,1,2,2,2
1,Did something happen to you?,0,0,1,0
2,"Thank you for taking part, I really appreciate...",2,1,2,2
3,I am grateful for your participation. I hope t...,1,2,2,2
4,"Thank you for being so open, I'm trying to det...",2,2,2,2
...,...,...,...,...,...
1095,Has this protocol made you feel better or worse?,0,0,0,0
1096,"Brilliant, that's always nice to hear. Would y...",1,1,2,1
1097,Do you always criticise yourself if anything g...,0,0,0,0
1098,Do you feel like you have let someone down?,0,0,0,0


In [9]:
df = df[df['empathy_level'].isna() == False]

In [11]:
df.columns = ['i', 'level_0', 'level_1', 'text', '-', '-2', 'empathy_level']

In [12]:
df

,i,level_0,level_1,text,-,-2,empathy_level
0,0,0,A01,How do you feel?,NaN,NaN,1.0
1,64,1,A01,How are you feeling?,NaN,NaN,1.0
2,128,2,A01,How are you feeling today?,NaN,NaN,1.0
3,192,3,A01,How are you doing?,NaN,NaN,1.0
4,256,4,A01,How are you feeling right now?,NaN,NaN,1.0
...,...,...,...,...,...,...,...
255,530,8,B10,Congratulations! You have successfully develop...,NaN,NaN,1.0
256,594,9,B10,Good job! It is possible you have shown tender...,NaN,NaN,1.0
257,658,10,B10,"Well done! You have shown tender compassion, t...",NaN,NaN,2.0
258,722,11,B10,Congratulations! You tend to express tender co...,NaN,NaN,0.0


## LISA Dataset

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [41]:
df = pd.read_csv("data/compassion_energy/empathy_labelled_ds.csv")

In [42]:
df

,response,annotator1_score,annotator2_score,annotator3_score,empathy_score
0,When you've selected the protocol you want to ...,1,2,2,2
1,Did something happen to you?,0,0,1,0
2,"Thank you for taking part, I really appreciate...",2,1,2,2
3,I am grateful for your participation. I hope t...,1,2,2,2
4,"Thank you for being so open, I'm trying to det...",2,2,2,2
...,...,...,...,...,...
1095,Has this protocol made you feel better or worse?,0,0,0,0
1096,"Brilliant, that's always nice to hear. Would y...",1,1,2,1
1097,Do you always criticise yourself if anything g...,0,0,0,0
1098,Do you feel like you have let someone down?,0,0,0,0


In [43]:
X = df['response']
y = df['empathy_score']

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=0)

## Model

In [65]:
import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

In [66]:
# Import our models. The package will take care of downloading the models automatically
tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")
model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")

In [67]:
texts = list(X_train)
texts_test = list(X_test)

In [68]:
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Get the embeddings
with torch.no_grad():
    embeddings = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output

In [69]:
inputs_test = tokenizer(texts_test, padding=True, truncation=True, return_tensors="pt")

# Get the embeddings
with torch.no_grad():
    embeddings_test = model(**inputs_test, output_hidden_states=True, return_dict=True).pooler_output

In [70]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(embeddings, y_train)
y_pred = clf.predict(embeddings_test)

/Users/weijiechua/miniforge3/envs/gen/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [71]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.62      0.67       240
           1       0.53      0.65      0.59       256
           2       0.81      0.75      0.78       274

    accuracy                           0.68       770
   macro avg       0.69      0.68      0.68       770
weighted avg       0.69      0.68      0.68       770



## Train One more layer in front

In [74]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [77]:
class ClassificationModel(nn.Module):
    def __init__(self, tokenizer, pt_model):
        self.tokenizer = tokenizer
        self.pt_model = pt_model
        self.fc = nn.Linear(768, 2)
        
    def predict(self, x):
        self.pt_model.eval()
        with torch.no_grad():
            prob = self.forward(x)
            return F.softmax(prob, dim=-1)
        
    def forward(self, x):
        inputs = self.tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
        embeddings = self.pt_model(**inputs, output_hidden_states=True, return_dict=True).pooler_output
        output = self.fc(embeddings)
        return output

In [80]:
cm = ClassificationModel(tokenizer, model)

AttributeError: cannot assign module before Module.__init__() call